In [6]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE  # Make sure to install imblearn if you haven't
from imblearn.pipeline import make_pipeline as make_pipeline_imb  # For integrating SMOTE with sklearn pipeline
import pickle

# Assuming necessary NLTK downloads are done
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

file_path = 'dataset.csv'
df = pd.read_csv(file_path, usecols=['par_id', 'paragraph', 'has_entity', 'category'])
df = df.dropna()
df['category'] = df['category'].str.lower()
df = df[df['has_entity'] != 'data missing']

def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['cleaned_paragraph'] = df['paragraph'].apply(clean_text)
df['tokens'] = df['cleaned_paragraph'].apply(nltk.word_tokenize)

# Preparing Data for Modeling
X = df['cleaned_paragraph']
y = df['category']
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# Splitting the data
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X, y_encoded, test_size=0.2, stratify=y_encoded)

# Adjusting for imbalance with SMOTE and building the text classification pipeline
smote_pipeline = make_pipeline_imb(
    CountVectorizer(stop_words='english'),
    TfidfTransformer(),
    SMOTE(random_state=42),
    MultinomialNB(alpha=.01)
)

# Fitting the model on the training data
smote_pipeline.fit(X_train, y_train_encoded)

# Predicting on new data
docs_new = [""]
predicted = smote_pipeline.predict(docs_new)
predicted_category = encoder.inverse_transform(predicted)

print(predicted_category)

model_extn = '.pkl'
model_name = 'model_cat_t1' + model_extn

# Serialization
with open(model_name, 'wb') as f:
    pickle.dump(smote_pipeline, f)

# Deserialization and Prediction with the loaded model
with open(model_name, 'rb') as f:
    loaded_model = pickle.load(f)
predicted = loaded_model.predict(docs_new)
print(encoder.inverse_transform(predicted))

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fe55c7d7f70>
Traceback (most recent call last):
  File "/opt/conda/envs/COMP1804-AML/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/opt/conda/envs/COMP1804-AML/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/opt/conda/envs/COMP1804-AML/lib/py

['biographies' 'movies about artificial intelligence' 'biographies']
['biographies' 'movies about artificial intelligence' 'biographies']
